In [1]:
import os
import sys
import warnings
import pandas 
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.drop("link", axis=1, inplace = True)

In [4]:
df = df[0:20000]
df.drop_duplicates(subset="song", inplace=True)
df.reset_index(drop = True, inplace = True)

In [5]:
df.rename(columns = {"artist":'Artist','song':'Song','text':'Lyrics'},inplace = True)

In [6]:
df['Combined_features'] = df['Artist']+" "+df['Song']+" "+df['Lyrics']

In [7]:
import re
import nltk
from nltk.corpus import stopwords
def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

In [8]:
df['Combined_features'] = df['Combined_features'].apply(func = cleaning)

In [9]:
tfv = TfidfVectorizer(max_features = 20000)

In [10]:
tfv_matrix = tfv.fit_transform(df["Combined_features"])

In [11]:
cosine_sim = cosine_similarity(tfv_matrix)

In [12]:
song_user_likes = "Hope"
song_index = df[df.Song == song_user_likes].index[0]

In [13]:
similar_songs = list(enumerate(cosine_sim[song_index]))

In [14]:
similar_song_sorted = sorted(similar_songs,key = lambda x:x[1],reverse = True)

In [15]:
def recommended_songs(song_user_likes):
    if song_user_likes in df['Song'].values:
        song_index = df[df['Song'] == song_user_likes].index[0]
        similar_songs = list(enumerate(cosine_sim[song_index]))
        similar_songs_sorted = sorted(similar_songs, key=lambda x: x[1], reverse=True)
        recommended_songs = []

        for i in range(1, len(similar_songs_sorted)):
            song_index = similar_songs_sorted[i][0]
            recommended_songs.append(df['Song'].iloc[song_index])

        return recommended_songs
    else:
        return "Song not found in the dataset"


In [17]:
recommended_songs("vcvc")

'Song not found in the dataset'

In [23]:
import pickle
pickle.dump(cosine_sim,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))